# Applied Data Science Week 3 Project

## Scrape the wikipedia page to get postal codes of Canada into a data frame
> *https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*

In [14]:
# !pip install beautifulsoup4
# !pip install wget

In [76]:
from bs4 import BeautifulSoup
import pandas as pd
# import wget
import requests

In [16]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [86]:
parsed_html = BeautifulSoup(page.content, 'html.parser')
table=parsed_html.body.find('table', attrs={'class':'wikitable'})

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# print(table.tbody.children)
for trow in table.tbody.children:
    td = trow.extract().find_all("td")
    if (len(td) > 1):
        neighborhoods = neighborhoods.append({'PostalCode': td[0].string.strip("\n"),
                                            'Borough': td[1].string.strip("\n"),
                                          'Neighborhood': td[2].string.strip("\n")}, ignore_index=True)

neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Now clean up the dataframe

In [122]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 10 boroughs and 103 neighborhoods.


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Verify there are no "Not assigned" cells in Neighborhood

In [126]:
df = neighborhoods
df[df.Neighborhood == "Not assigned"].head()

,PostalCode,Borough,Neighborhood


In [117]:
# df = neighborhoods.groupby(["PostalCode"]).count()
# df = df.reset_index()
# df.head()

# # df.describe()
# # df[df.PostalCode == "M1A"]

,PostalCode,Borough,Neighborhood


In [127]:

print("The shape of the dataframe is {}".format(df.shape))

The shape of the dataframe is (103, 3)
